#### 설명  

K-means  

- 관측값들을 가장 비슷한 그룹(클러스터)으로 자동 분류하는 대표적인 비지도 학습 알고리즘  
- “거리 기반으로 비슷한 것끼리 묶는다”는 것  

---

개념  

- K-means는 데이터를 K개의 그룹으로 자동 분류하는 알고리즘입니다.  
- 'K' = 만들고 싶은 군집(cluster)의 개수  
- 'means' = 각 군집의 중심(mean, centroid)을 이용해 데이터 분류  

즉,군집 중심을 임의로 설정하고 > 각 데이터를 가장 가까운 중심에 배정한 뒤 > 중심을 다시 계산하여 갱신 > 이 과정을 반복해 최적의 분류를 찾는다.  

---

작동 방식  

|단계|제목|설명|
|---|---|---|
|1|K개의 중심점(centroids)을 초기 설정|처음에는 랜덤하게 선택됩니다.|
|2|각 데이터가 가장 가까운 중심점(거리 기준)에 할당|예: A 전공은 1번 중심에 더 가까우므로 클러스터 1|
|3|중심점 다시 계산|각 클러스터에 속한 데이터들의 평균을 새 중심점으로 설정합니다.|
|4|중심점 변화가 없을 때까지 반복|더 이상 바뀌지 않으면 군집이 확정됩니다.|

---

4. K-means의 장점과 한계  

장점  

- 매우 빠르고 계산 효율적
- 고차원 데이터에서도 잘 작동
- 해석이 직관적(유사한 것끼리 묶는 구조)

한계  

- K(군집 개수)를 직접 지정해야 함
- 원형(구형) 구조의 데이터에 강함, 복잡한 모양은 약함
- 이상치(outlier)에 민감
- 변수 스케일 차이에 민감 → 그래서 표준화(StandardScaler) 필수

---

5. 적절한 K 값을 찾는 방법  

(1) Elbow Method  
- K를 1~10 사이의 값으로 바꿔가며 학습  
- 각 K에 대해 군집 내 제곱합(WCSS) 계산  
- 그래프가 꺾이는 지점(팔꿈치 모양)을 최적 K로 선택  
- 왜 꺾이는 지점인가?  
- K가 증가하면 WCSS는 계속 줄어듦  
- 그러나 일정 지점부터는 감소폭이 둔화됨  
- “추가로 K를 늘려봤자 개선이 거의 없는 지점”이 최적  


(2) Silhouette Score (실루엣 계수)  

- 군집이 얼마나 잘 분리되고, 군집 내부가 얼마나 촘촘한지를 평가하는 지표입니다.  
- 값 범위: -1 ~ 1  
- 1에 가까울수록 군집 품질 좋음  
- 0은 경계에 있음  
- 음수는 잘못된 군집화  
- K를 2~10으로 바꿔가며 silhouette score 측정 → 가장 점수가 높은 K를 선택.  

(3) 실무에서는 어떻게 결정하나?  

- 엘보우 기법으로 후보 K 2~4개 추림  
- 실루엣 점수로 최종 선택  
- 이 방식이 가장 안정적입니다.  
- 특히 교육·심리·사회 데이터에서는 K=3~5 사이에서 결정되는 경우가 많습니다.  

--- 

(1-1) 엘보우가 꺾이는 지점을 자동으로 선택하는 방법  

|No|방법|설명|
|---|---|---|
|1|Kneedle Locator|엘보우는 결국 “곡선이 가장 많이 꺾이는 지점”을 찾는 문제이므로, Kneedle 알고리즘은 **곡률(curvature)**를 계산하여 자동으로 엘보우를 찾아줍니다.|
|2|감소율(기울기) 변화율 기반 방식|기울기의 변화율이 가장 큰 지점이 최적 K, Δk​=WCSS(k−1)−WCSS(k)|
|3|상대적 감소 비율(Ratio-of-Change)|WCSS(k)와 WCSS(k+1)의 차이를 WCSS(k)로 나눈 값이 가장 큰 지점이 최적 K.<br>$r_{k} = \frac{WCSS(k)}{WCSS(k-1)}$|

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.metrics import silhouette_score

# dataset
data_path = data_dir + "./전공X교과별_핵심역량_연도_학기.csv"
df = pd.read_csv(data_path, index_col=0, encoding="cp949")

# k-means clustering
def kmeans_clustering(df, k, features, visualization:bool=True):
    df_temp = df.copy()
    X = df_temp[features]
    scaler = StandardScaler()
    X_scaled = scaling(X)
    # Standard Scaler : 각 변수의 평균을 0, 표준편차를 1로 맞추는 표준화(Standardization) 도구
    kmeans = KMeans(n_clusters=k, random_state=42)
    df_temp["cluster"] = kmeans.fit_predict(X_scaled)
    cluster_profile = df_temp.groupby("cluster").mean()
    if visualization:
        visualization_2d(X_scaled, df_temp["cluster"])
    return df_temp, cluster_profile

# elbow method
def elbow_method(df, features, max_range:int=10):
    df_scaled = scaling(df[features])
    result = []
    for i in range(max_range):
        kmeans = KMeans(n_clusters=i+1, random_state=42)
        kmeans.fit(df_scaled)
        result.append(kmeans.inertia_)
        print(f"ELBOW -- K={i+1}: {kmeans.inertia_}") # WCSS 값(군집 내 제곱합)
    kl = KneeLocator(range(1, max_range+1), result, curve="convex", direction="decreasing")
    print(f"ELBOW -- Optimal K: {kl.elbow}")
    return kl.elbow

# silhouette method
def silhouette_method(df, features, min_range:int=1, max_range:int=10):
    best_k = 0
    best_score = 0
    for k in range(min_range, max_range+1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)
        print(f"SILHOUETTE -- K={k}, Score={score}")
        if score > best_score:
            best_k = k
            best_score = score 
    print(f"SILHOUETTE -- Optimal K: {best_k}")
    return best_k

# scaling
def scaling(df):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    return df_scaled

# 2d visualization
def visualization_2d(X_scaled, clusters):
    pca = PCA(n_components=2)
    pca_data = pca.fit_transform(X_scaled)

    plt.scatter(pca_data[:,0], pca_data[:,1], c=clusters)
    plt.title("K-means Clustering (PCA 2D Visualization)")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()
